# Project 2 - Customer loans in Finance

This file serves as a tool for myself to revisit what I have done and the things that I have learnt.

## Milestone 1-2: Initialise & run a class to extract the data 

**1. Initialise the class**

- Created a Python file to contain code for extraction - **db_utils.py**
- Created a Class **RDSDatabase** which will be used for the extraction

**2. Store Database Credentials**

- Created a credentials.yaml file to store the database credentials provided by AiCore
- Created a .gitignore file to keep the credentials secure and prevent them from being pushed to GitHub: 

    1. Create .gitignore file
        - git init > git touch .gitignore > git nano .gitignore > 
    2. Add in credentials.yaml
        - git add .gitignore 
    3. Commit to Github
        - git commit -m "Adding .gitignore to GitHub" > git push origin main
     

**2. Load credentials**

In [ ]:
import yaml

def load_credentials(filepath: str) -> dict:
    with open(filepath, "r") as f:
        credentials = yaml.safe_load(f)
    

Add in ErrorHandling controls

In [ ]:
import yaml

def load_credentials(filepath: str) -> dict:
    try:
        with open(filepath, "r") as f:
            credentials = yaml.safe_load(f)
            return credentials
    except ExceptionError as e:
        print(f"Error loading credentials {e}")
        return {}

**3. Initialise RDSDatabase Connector**

- Initilising RDSDataseConnector taking the dictionary of credentials from above as a parameter
- Setting "*self.engine = None*". Initilising it this way means that I am ensuring that the attributed are used only when they have valid values. It will be then set to a SQLAlchelmy engine object later. 

In [ ]:
class RDSDatabaseConnector:
    def __init__(self, credentials: dict):
        self.credentials = credentials
        self.engine = None

**4. Initialise SQLAlchemy Engine**

- Defined method in RDSDatabaseConnector to set the engine to the SQLAlchemy Engine

In [ ]:
    def initialise_engine(self):
        '''
        Initialises a SQLAlchemy engine using the provided credentials
        '''
        try:
            engine_url = (f"postgresql://
            {self.credentials['RDS_USER']}:{self.credentials['RDS_PASSWORD']}@{self.credentials['RDS_HOST']}/{self.credentials['RDS_DATABASE']}")
            self.engine=create_engine(engine_url)
            print("SQLAlchemy engine initialized successfully.")
        except Exception as e:
            print(f"Error initializing SQLAlchemy engine: {e}")

**5. Extract data**
- Created a method to extract data from the RDS Database and return it as a Pandas DataFrame

In [ ]:
def extract_data(self, query:str) -> pd.DataFrame:
    if self.engine is None: 
        raise ValueError ("Engine is not initialised. Call initialise_engine() first")
    return pd.read_sql(query,self.engine)


**5. Create function to save the extracted data to a local file**

In [ ]:
def save_to_csv (self, data: pd.DataFrame, filename: str):
    data.to_csv(filename, index=False)

**6. Disconnect**

In [2]:
def disconnect():
    if self.engine:
        self.engine.dispose()
        print("SQLAlchemy engine connection is closed")
    else:
        print("No active connection to close.")

**7. Call the method** 
- To ensure that this code is only run when called upon I included the line if __name__ = "__main__" 
- Then called the method to connect to the database and disconnect once finished saving the data to the csv.

In [ ]:
if __name__ == "__main__":
    credentials = load_credentials("credentials.yaml")

    connector = RDSDatabaseConnector(credentials)
    connector.initialise_engine()
    
    query = "SELECT * FROM loan_payments"
    data = connector.extract_data(query)

    if not data.empty:
        connector.save_to_csv(data,"loan_payments.csv")
    
    connector.disconnect()


## Milestone 3: Exploratory Data Analysis (EDA)

This milestone is set to gain a deeper understanding of the data and identify any patterns which might exist. I'll be: 
- Reviewing the data to identify any issues, such as missing or incorrectly formatted data. 
- Applying statistical techniques to gain insight on the data's distribution and apply visualisation techniques to identify patterns or trends in the data. 

#### **Task 1**

**Convert columns to the correct format within DataTransform Class** 

Are there any columns in the exisiting df that need amending? 

From the original data *df = pd.df = pd.read_csv("loan_payments.csv) > print(df.types)* I convert the following: 
- **term** = currently an object so convert to numberical but converting to an integer representing the number of months.
- **issue_date, earliest_credit_line, last_payment_date, next_payment_date, last_credit_pull_date** = need to convert to datetime.
- **employment_length** = convert to an integer, for <1 and 10+ change to 0 and 10 respectively. 
- **loan_status** = As it contains a limited number of unique values I convert to category.

For employment date I need to extract the number from the full details given in the column :


In [ ]:
df[column] = df[column].str.replace('< 1 year', '0')
df[column] = df[column].str.replace('10+ years', '10')
df[column] = df[column].str.extract(r'(\d+)') #Explaination below
df[column] = df[column].astype(float)

For df[column].str.extract(r'(\d+)')
- **r** indicates that the string is a raw string, which means that the backslashes are treated as literal characters and not as escape characters
- **\d** matches any 0-9 digit
- **+** means "one or more" of the preceeding element in this case is digits

So **(r'(\d+)')** matches one or more digits in the string and captures them as a group. The **str.extract** method returns a df with the extracted digits. 

**Defined DataTransform Class and opened new ipynb to ensure all analysis is in one place**

- This .ipynb is still used for my own personal understanding of what I learnt throughtout this project.

- Analysis though is now found on loan_portfolio_analysis.ipynb

For those columns that needed changing to datetime, if no date_format is provided, the method uses the default parsing behaviour of pd.to_datetime(). 

By explicitly stating the date format that we want to use we avoid potential errors that may arise later on. 

In [ ]:
  def convert_to_datetime(self, column: str, date_format: str = None) -> pd.DataFrame:
        if date_format:
            self.df[column] = pd.to_datetime(self.df[column], format=date_format)
        else: 
            self.df[column] = pd.to_datetime(self.df[column])
        return self.df

Calling the method we have to specify the format wanted. In the data these columns are presented by 'Jan-2021 or May-2025' etc so abbreviated month name and total year. 

In Python to format this you have many options: 
- %B = Full month name 
- %b = abbreviated month name
- %Y = four-digit year
- %y = two-digit year

With multiple columns to be tranformed you can do all in one go via: 

In [ ]:
    def convert_multiple_to_datetime(self, columns: list, date_format: str = None) -> pd.DataFrame:
        for column in columns: 
            if date_format:
                self.df[column] = pd.to_datetime(self.df[column], format=date_format)
            else: 
                self.df[column] = pd.to_datetime(self.df[column])
        return self.df

Then you call it with which will enable all the columns called out will be converted

In [ ]:
columns_to_convert_to_datetime = [
    'issue_date',
    'earliest_credit_line',
    'last_payment_date',
    'next_payment_date',
    'last_credit_pull_date'
]
transformer.convert_multiple_to_datetime(
    columns_to_convert_to_datetime, date_format='%b-%Y'
    )

#### **Task 2**

- Describe all columns in the DataFrame to check their data types
- Extract statistical values: median, standard deviation and mean from the columns and the DataFrame
- Count distinct values in categorical columns
- Print out the shape of the DataFrame
- Generate a count/percentage count of NULL values in each column
- Any other methods you may find useful.

Here I set up a new .py file **dataframe_info.py** to do host this code. It is best practice to seperate classes to ensure ease of following.